In [1]:
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._


val lift_df = Seq(
    (1,300),
    (2,350)
).toDF("id", "capacity_kg")


val lift_passengers_data = Seq(
    Row("Rahul",85,1),
    Row("Adarsh",73,1),
    Row("Riti",95,1),
    Row("Viraj",80,1),
    Row("Vimal",83,2),
    Row("Neha",77,2),
    Row("Priti",73,2),
    Row("Himanshi",85,2)
)

val lift_passengers_schema = StructType(Array(
  StructField("passenger_name", StringType),
  StructField("weight_kg", IntegerType),
  StructField("lift_id", IntegerType)
))

val lift_passengers_rdd = spark.sparkContext.parallelize(lift_passengers_data)
val lift_passengers_df = spark.createDataFrame(lift_passengers_rdd, lift_passengers_schema)


lift_df.show(false)
lift_passengers_df.show(false)

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.5:4040
SparkContext available as 'sc' (version = 3.4.1, master = local[*], app id = local-1716989732517)
SparkSession available as 'spark'


+---+-----------+
|id |capacity_kg|
+---+-----------+
|1  |300        |
|2  |350        |
+---+-----------+

+--------------+---------+-------+
|passenger_name|weight_kg|lift_id|
+--------------+---------+-------+
|Rahul         |85       |1      |
|Adarsh        |73       |1      |
|Riti          |95       |1      |
|Viraj         |80       |1      |
|Vimal         |83       |2      |
|Neha          |77       |2      |
|Priti         |73       |2      |
|Himanshi      |85       |2      |
+--------------+---------+-------+



import org.apache.spark.sql.Row
import org.apache.spark.sql.types._
lift_df: org.apache.spark.sql.DataFrame = [id: int, capacity_kg: int]
lift_passengers_data: Seq[org.apache.spark.sql.Row] = List([Rahul,85,1], [Adarsh,73,1], [Riti,95,1], [Viraj,80,1], [Vimal,83,2], [Neha,77,2], [Priti,73,2], [Himanshi,85,2])
lift_passengers_schema: org.apache.spark.sql.types.StructType = StructType(StructField(passenger_name,StringType,true),StructField(weight_kg,IntegerType,true),StructField(lift_id,IntegerType,true))
lift_passengers_rdd: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = ParallelCollectionRDD[0] at parallelize at <console>:50
lift_passengers_df: org.apache.spark.sql.DataFrame = [passenger_name: string, weight_kg: int ... 1 more field]


In [6]:
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions.{collect_list,concat_ws}

lift_df.join(lift_passengers_df, $"id" === $"lift_id", "inner"
            ).withColumn("running_sum", sum($"weight_kg").over(Window.partitionBy($"lift_id").orderBy($"weight_kg".asc))
                        ).where($"running_sum" <= $"capacity_kg"
                               ).groupBy($"id").agg(concat_ws(",",collect_list($"passenger_name"))
                                                   ).as("passenger_names").show(false)

+---+------------------------------------------+
|id |concat_ws(,, collect_list(passenger_name))|
+---+------------------------------------------+
|1  |Adarsh,Viraj,Rahul                        |
|2  |Priti,Neha,Vimal,Himanshi                 |
+---+------------------------------------------+



import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions.{collect_list, concat_ws}
